<a href="https://colab.research.google.com/github/Estafilococo/satellite_passes/blob/main/satellite_passes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.5/234.5 kB 11.2 MB/s eta 0:00:00


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests
from skyfield.api import wgs84, load
import math
import datetime
import pytz


OBS_LAT = 40.481
OBS_LON = -3.364
TLE_URL = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
TLE_LOCAL_FILE = 'satellites.tle'

MIN_ELEV_DEG = 70.0
MAX_ELEV_DEG = 90.0

LOCAL_TZ = pytz.timezone('Europe/Madrid')

SAT_INFO = {


    "GPS (Constelación NAVSTAR)": {
        "descripcion": "Constelación de posicionamiento global de EE.UU. (GPS).",
        "frecuencia": "L1: 1575.42 MHz, L2: 1227.60 MHz",
        "modo": "CDMA"
    },
    "GLONASS": {
        "descripcion": "Constelación de posicionamiento global de Rusia.",
        "frecuencia": "1602 MHz, 1246 MHz (FDMA)",
        "modo": "FDMA"
    },
    "Galileo": {
        "descripcion": "Constelación europea de posicionamiento (ESA).",
        "frecuencia": "E1: 1575.42 MHz, E5: 1191.795 MHz, E6...",
        "modo": "CDMA"
    },
    "BeiDou/Compass": {
        "descripcion": "Constelación china de navegación y posicionamiento.",
        "frecuencia": "B1I: 1561.098 MHz, B2I: 1207.14 MHz...",
        "modo": "CDMA"
    },
    "NavIC (IRNSS)": {
        "descripcion": "Sistema de navegación regional de la India (ISRO).",
        "frecuencia": "L5 (1176.45 MHz) y S-band (2492.028 MHz)",
        "modo": "CDMA"
    },
    "QZSS": {
        "descripcion": "Quasi-Zenith Satellite System, Japón (JAXA).",
        "frecuencia": "L1, L2, L5, L6",
        "modo": "CDMA"
    },

    "Landsat 8": {
        "descripcion": "Satélite de observación terrestre (NASA/USGS).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Landsat 9": {
        "descripcion": "Última generación Landsat (NASA/USGS).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Sentinel-1": {
        "descripcion": "Radar de apertura sintética (SAR) de la ESA.",
        "frecuencia": "C-band",
        "modo": "SAR"
    },
    "Sentinel-2": {
        "descripcion": "Óptico multiespectral de alta resolución (ESA).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Sentinel-3": {
        "descripcion": "Oceanografía y monitoreo global (ESA).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Sentinel-4": {
        "descripcion": "Instrumento geoestacionario de vigilancia atmosférica (ESA).",
        "frecuencia": "Desconocido (carga en satélite Meteosat)",
        "modo": "Espectrómetro"
    },
    "Sentinel-5": {
        "descripcion": "Monitoreo de gases atmosféricos (ESA).",
        "frecuencia": "Desconocido (carga en satélite MetOp)",
        "modo": "Espectrómetro"
    },
    "Sentinel-6": {
        "descripcion": "Altímetro de precisión para océanos (ESA/NASA).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "GOES-16": {
        "descripcion": "Satélite meteorológico geoestacionario de EE.UU.",
        "frecuencia": "L-band y S-band (telemetría, GVAR)",
        "modo": "No aplica"
    },
    "GOES-17": {
        "descripcion": "Satélite meteorológico geoestacionario de EE.UU.",
        "frecuencia": "L-band y S-band (telemetría, GVAR)",
        "modo": "No aplica"
    },
    "Meteosat": {
        "descripcion": "Serie europea de satélites meteorológicos (EUMETSAT).",
        "frecuencia": "L-band, HRIT/LRIT",
        "modo": "No aplica"
    },
    "Himawari-8": {
        "descripcion": "Satélite meteorológico de Japón (JMA).",
        "frecuencia": "HRIT/LRIT en banda L",
        "modo": "No aplica"
    },
    "Himawari-9": {
        "descripcion": "Satélite meteorológico de Japón (JMA).",
        "frecuencia": "HRIT/LRIT en banda L",
        "modo": "No aplica"
    },
    "Feng Yun (serie)": {
        "descripcion": "Satélites meteorológicos chinos (CMA).",
        "frecuencia": "Varios (L-band, X-band)",
        "modo": "No aplica"
    },
    "CARTOSAT": {
        "descripcion": "Satélites de observación de la India (ISRO).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },

    "Intelsat (serie)": {
        "descripcion": "Constelación geoestacionaria internacional (Intelsat).",
        "frecuencia": "C/Ku/Ka-band",
        "modo": "Telecom"
    },
    "Eutelsat (serie)": {
        "descripcion": "Satélites de comunicaciones europeos (Eutelsat).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "SES (serie)": {
        "descripcion": "Satélites de SES (Luxemburgo).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "Iridium NEXT": {
        "descripcion": "Constelación de comunicaciones satelitales (EE.UU.).",
        "frecuencia": "L-band para telefonía, Ka-band para inter-satélite",
        "modo": "Telefonía/datos"
    },
    "Starlink": {
        "descripcion": "Constelación de internet banda ancha (SpaceX).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Datos"
    },
    "OneWeb": {
        "descripcion": "Constelación de internet global (Reino Unido/India).",
        "frecuencia": "Ku-band",
        "modo": "Datos"
    },

    "Hubble Space Telescope": {
        "descripcion": "Telescopio espacial conjunto NASA/ESA.",
        "frecuencia": "S-band (comandos), Ku-band (datos)",
        "modo": "Telemetría"
    },
    "James Webb Space Telescope": {
        "descripcion": "Telescopio espacial NASA/ESA/CSA.",
        "frecuencia": "Ka-band ~25.9 GHz (downlink), S-band (uplink)",
        "modo": "Telemetría"
    },
    "GAIA": {
        "descripcion": "Observatorio astrométrico de la ESA.",
        "frecuencia": "X-band / Ka-band",
        "modo": "Telemetría"
    },
    "XMM-Newton": {
        "descripcion": "Observatorio de rayos X de la ESA.",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },
    "Chandra X-ray Observatory": {
        "descripcion": "Observatorio de rayos X de la NASA.",
        "frecuencia": "S-band (comandos), K-band (datos)",
        "modo": "Telemetría"
    },
    "TESS": {
        "descripcion": "Satélite de censo de exoplanetas (NASA).",
        "frecuencia": "Ka-band (datos)",
        "modo": "Telemetría"
    },

    "Estación Espacial Internacional (ISS)": {
        "descripcion": "Estación espacial internacional (EE.UU./Rusia/etc.).",
        "frecuencia": "145.800 MHz (downlink FM), 437.550 MHz (APRS)",
        "modo": "FM/APRS"
    },
    "Tiangong": {
        "descripcion": "Estación Espacial China (CSS).",
        "frecuencia": "Posibles tests en 145.825 MHz",
        "modo": "FM"
    },

    "KH-11 KENNEN (serie)": {
        "descripcion": "Satélites espía de EE.UU.",
        "frecuencia": "Clasificado",
        "modo": "Reconocimiento"
    },
    "Helios (serie)": {
        "descripcion": "Satélites de observación militar de Francia.",
        "frecuencia": "Clasificado",
        "modo": "Reconocimiento"
    },
    "SAR-Lupe": {
        "descripcion": "Constelación alemana de reconocimiento (SAR).",
        "frecuencia": "X-band (posiblemente)",
        "modo": "SAR militar"
    },
    "COSMO-SkyMed": {
        "descripcion": "Constelación italiana de observación radar.",
        "frecuencia": "X-band (SAR)",
        "modo": "Reconocimiento"
    },
    "Yaogan (serie)": {
        "descripcion": "Satélites chinos de reconocimiento (SAR/óptico).",
        "frecuencia": "Clasificado",
        "modo": "Reconocimiento"
    },
    "Ofek (serie)": {
        "descripcion": "Satélites espía de Israel.",
        "frecuencia": "Clasificado",
        "modo": "Reconocimiento"
    },

    "OCO-2": {
        "descripcion": "Observatorio de carbono en órbita (NASA).",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },
    "CALIPSO": {
        "descripcion": "Satélite NASA/CNES para estudio de nubes y aerosoles.",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "DMSP (serie)": {
        "descripcion": "Programa de satélites meteorológicos de EE.UU. (militar).",
        "frecuencia": "Clasificado / S-band",
        "modo": "Telemetría"
    },
    "CloudSat": {
        "descripcion": "Radar de nubes (NASA).",
        "frecuencia": "X-band (downlink), 94 GHz (radar)",
        "modo": "Telemetría"
    },

    "AsiaSat (serie)": {
        "descripcion": "Satélites de comunicaciones de AsiaSat (Hong Kong).",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "MEASAT (serie)": {
        "descripcion": "Satélites de Malasia (Measat Satellite Systems).",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Arabsat (serie)": {
        "descripcion": "Flota de satélites de la Liga Árabe.",
        "frecuencia": "C/Ku/Ka-band",
        "modo": "Telecom"
    },
    "Türksat (serie)": {
        "descripcion": "Satélites de telecomunicaciones de Turquía.",
        "frecuencia": "Ku-band",
        "modo": "Telecom"
    },
    "Sky Perfect JSAT": {
        "descripcion": "Operador satelital japonés (JSAT).",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Telstar (serie)": {
        "descripcion": "Satélites de Telesat (Canadá).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "DirecTV (serie)": {
        "descripcion": "Satélites de televisión directa (EE.UU.).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "Amazonas (serie)": {
        "descripcion": "Satélites de Hispasat (España/América Latina).",
        "frecuencia": "Ku/Ka-band",
        "modo": "Telecom"
    },
    "O3b (constelación)": {
        "descripcion": "Constelación MEO de SES para banda ancha.",
        "frecuencia": "Ka-band",
        "modo": "Datos"
    },
    "Yamal (serie)": {
        "descripcion": "Satélites de telecomunicaciones de Rusia (Gazprom).",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Badr (serie)": {
        "descripcion": "Satélites Arabsat (posición orbital 'Badr').",
        "frecuencia": "C/Ku-band",
        "modo": "Telecom"
    },
    "Chinasat (serie)": {
        "descripcion": "Satélites de comunicación chinos (ChinaSat).",
        "frecuencia": "C/Ku/Ka-band",
        "modo": "Telecom"
    },
    "Koreasat (serie)": {
        "descripcion": "Satélites de telecomunicaciones de Corea del Sur.",
        "frecuencia": "Ku-band",
        "modo": "Telecom"
    },
    "VSAT (serie)": {
        "descripcion": "Término genérico para Very Small Aperture Terminal.",
        "frecuencia": "Ku/Ka/C-band",
        "modo": "Telecom"
    },

    "RADARSAT Constellation": {
        "descripcion": "Constelación de Canadá (SAR).",
        "frecuencia": "C-band (SAR)",
        "modo": "SAR"
    },
    "TerraSAR-X": {
        "descripcion": "Satélite alemán de observación radar.",
        "frecuencia": "X-band (SAR)",
        "modo": "SAR"
    },
    "SPOT (serie)": {
        "descripcion": "Satélites de observación franceses (Airbus).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "ResourceSat": {
        "descripcion": "Serie india de observación (ISRO).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "KOMPSAT (serie)": {
        "descripcion": "Satélites de observación de Corea del Sur.",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Gaofen (serie)": {
        "descripcion": "Satélites chinos de alta resolución.",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "ALOS (serie)": {
        "descripcion": "Satélites japoneses de observación (JAXA).",
        "frecuencia": "L-band (SAR), X-band",
        "modo": "SAR/Óptico"
    },
    "TanDEM-X": {
        "descripcion": "Misión alemana (DLR) de interferometría con TerraSAR-X.",
        "frecuencia": "X-band (SAR)",
        "modo": "SAR"
    },
    "WorldView (serie)": {
        "descripcion": "Satélites de DigitalGlobe/Maxar (EE.UU.).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Planet Labs (constelación)": {
        "descripcion": "Constelación de cubesats (Dove, SuperDove, etc.).",
        "frecuencia": "UHF para telemetría, X-band downlink",
        "modo": "Telemetría"
    },
    "RapidEye (constelación)": {
        "descripcion": "Constelación alemana (BlackBridge, ahora Planet).",
        "frecuencia": "S/X-band",
        "modo": "Telemetría"
    },
    "SkySat (constelación)": {
        "descripcion": "Satélites de alta resolución de Planet.",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },
    "NigeriaSat": {
        "descripcion": "Satélites de observación de Nigeria (NASRDA).",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },
    "VNREDSat": {
        "descripcion": "Satélite de observación de Vietnam.",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },

    "Solar and Heliospheric Observatory (SOHO)": {
        "descripcion": "Observatorio solar ESA/NASA.",
        "frecuencia": "X-band (DSN)",
        "modo": "Telemetría"
    },
    "Spektr-RG": {
        "descripcion": "Observatorio ruso-alemán de rayos X.",
        "frecuencia": "X-band",
        "modo": "Telemetría"
    },
    "Chandrayaan-2": {
        "descripcion": "Misión lunar india (ISRO).",
        "frecuencia": "S/X-band",
        "modo": "Telemetría"
    },
    "BepiColombo": {
        "descripcion": "Sonda ESA/JAXA rumbo a Mercurio.",
        "frecuencia": "X-band, Ka-band",
        "modo": "Telemetría"
    },
    "Solar Orbiter": {
        "descripcion": "Sonda ESA/NASA para estudio del Sol.",
        "frecuencia": "X-band, algunos tests en Ka-band",
        "modo": "Telemetría"
    },
    "DAMPE": {
        "descripcion": "Exploración de partículas de materia oscura (China).",
        "frecuencia": "S/X-band",
        "modo": "Telemetría"
    },
    "Astrosat": {
        "descripcion": "Observatorio de la India (ISRO).",
        "frecuencia": "S-band (comandos), X-band (datos)",
        "modo": "Telemetría"
    },
    "ADITYA-L1": {
        "descripcion": "Misión solar india (ISRO).",
        "frecuencia": "S/X-band",
        "modo": "Telemetría"
    },
    "ExoMars Trace Gas Orbiter": {
        "descripcion": "Orbitador marciano ESA/Roscosmos.",
        "frecuencia": "X-band (telecom), UHF (relay con rover)",
        "modo": "Telemetría"
    },

    "MetOp (serie)": {
        "descripcion": "Satélites polares de EUMETSAT.",
        "frecuencia": "L-band, X-band",
        "modo": "Telemetría"
    },
    "NOAA (serie)": {
        "descripcion": "Satélites meteorológicos polares de EE.UU.",
        "frecuencia": "137 MHz (APT), L-band, etc.",
        "modo": "APT/HRPT"
    },
    "GEO-KOMPSAT": {
        "descripcion": "Satélites meteorológicos geoestacionarios de Corea.",
        "frecuencia": "L-band, etc.",
        "modo": "No aplica"
    },
    "INSAT (serie)": {
        "descripcion": "Satélites meteorológicos/telecom de India (ISRO).",
        "frecuencia": "C/Extended-C/Ku-band",
        "modo": "Telecom/meteorología"
    },
    "Electro-L (serie)": {
        "descripcion": "Satélites meteorológicos rusos (GEO).",
        "frecuencia": "HRIT/LRIT, etc.",
        "modo": "No aplica"
    },
    "FY-4 (serie)": {
        "descripcion": "Satélites meteorológicos chinos en GEO.",
        "frecuencia": "L-band (downlink), etc.",
        "modo": "No aplica"
    },
    "COSMIC-2 (constelación)": {
        "descripcion": "Constelación Taiwán/EE.UU. para radio-ocultación GNSS.",
        "frecuencia": "GNSS (L1/L2), S-band downlink",
        "modo": "Datos"
    },

    "AMSAT (serie)": {
        "descripcion": "Satélites de radioaficionados (diversos países).",
        "frecuencia": "Varias bandas VHF/UHF/SHF",
        "modo": "FM, SSB, CW, Digital"
    },
    "FUNcube (serie)": {
        "descripcion": "Satélites educativos de AMSAT-UK.",
        "frecuencia": "435 MHz (downlink), 145 MHz (uplink)",
        "modo": "SSB/FM"
    },
    "BY70 (serie)": {
        "descripcion": "Pequeños satélites chinos de radioaficionados.",
        "frecuencia": "VHF/UHF",
        "modo": "FM"
    },
    "LilacSat (serie)": {
        "descripcion": "Proyectos de la Universidad de Harbin, China.",
        "frecuencia": "VHF/UHF",
        "modo": "FM/Transpondedor digital"
    },
    "ES'HAIL-2": {
        "descripcion": "Qatar-OSCAR 100, primer satélite geoaficionado.",
        "frecuencia": "Uplink ~2.4 GHz, Downlink ~10.489 GHz",
        "modo": "SSB/CW/DATV"
    },

    "COSPAS-SARSAT": {
        "descripcion": "Sistema internacional de búsqueda y rescate.",
        "frecuencia": "406 MHz (balizas), LEO/GEO satélites",
        "modo": "SAR"
    },
    "GOES-R (componente SAR)": {
        "descripcion": "Satélites GOES con transpondedor de rescate.",
        "frecuencia": "406 MHz (balizas)",
        "modo": "SAR"
    },
    "Galileo (componente SAR)": {
        "descripcion": "SAR integrado en constelación Galileo.",
        "frecuencia": "406 MHz (balizas), L-band",
        "modo": "SAR"
    },
    "GLONASS (componente SAR)": {
        "descripcion": "Segmento de búsqueda y rescate en GLONASS.",
        "frecuencia": "406 MHz (balizas)",
        "modo": "SAR"
    },

    "Project Kuiper": {
        "descripcion": "Constelación de internet (Amazon).",
        "frecuencia": "Ka-band (planeado)",
        "modo": "Datos"
    },
    "AST SpaceMobile (BlueWalker 3)": {
        "descripcion": "Satélite demo de telefonía directa al móvil.",
        "frecuencia": "Cellular/4G/5G, bandas satelitales",
        "modo": "Datos/Telefonía"
    },
    "Capella Space": {
        "descripcion": "Constelación comercial SAR (EE.UU.).",
        "frecuencia": "X-band",
        "modo": "SAR"
    },
    "ICEYE": {
        "descripcion": "Constelación comercial SAR (Finlandia).",
        "frecuencia": "X-band (SAR)",
        "modo": "SAR"
    },
    "GHGSat": {
        "descripcion": "Monitoreo de gases de efecto invernadero (Canadá).",
        "frecuencia": "X-band (downlink)",
        "modo": "Telemetría"
    },
    "Satellogic (NewSat)": {
        "descripcion": "Constelación argentina de observación.",
        "frecuencia": "U/VHF TLM, X-band downlink",
        "modo": "Telemetría"
    },
    "Astroscale (ELSA-d)": {
        "descripcion": "Demostrador de eliminación de desechos espaciales.",
        "frecuencia": "S/X-band",
        "modo": "Demostración"
    },
}

def download_tle(url, output_file):
    print(f"Descargando TLE desde: {url}")
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        with open(output_file, 'wb') as f:
            f.write(resp.content)
        print(f"TLE descargado y guardado en '{output_file}'.\n")
    except Exception as e:
        print(f"Ocurrió un error al descargar el TLE: {e}")

def get_inclination_deg(satellite):
    return satellite.model.inclo * (180.0 / math.pi)

def find_next_n_mondays(n=3, tz=LOCAL_TZ):

    now_local = datetime.datetime.now(tz)
    current_wd = now_local.weekday()
    days_until_monday = (0 - current_wd) % 7
    first_monday = now_local + datetime.timedelta(days=days_until_monday)

    first_monday = first_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    return [first_monday + datetime.timedelta(weeks=i) for i in range(n)]

def main():

    download_tle(TLE_URL, TLE_LOCAL_FILE)

    print("Cargando TLE desde fichero local...\n")
    satellites = load.tle_file(TLE_LOCAL_FILE)
    print(f"Se han cargado {len(satellites)} satélites.\n")

    observer_location = wgs84.latlon(OBS_LAT, OBS_LON)

    next_mondays_local = find_next_n_mondays(n=3, tz=LOCAL_TZ)

    monday_windows_utc = []
    ts = load.timescale()
    for monday_local in next_mondays_local:
        start_local = monday_local.replace(hour=18, minute=0, second=0, microsecond=0)
        end_local   = monday_local.replace(hour=21, minute=0, second=0, microsecond=0)

        start_utc = start_local.astimezone(pytz.utc)
        end_utc   = end_local.astimezone(pytz.utc)

        t_start = ts.from_datetime(start_utc)
        t_end   = ts.from_datetime(end_utc)
        monday_windows_utc.append((t_start, t_end))

    resultados = []

    for sat in satellites:
        sat_name = sat.name
        incl_deg = get_inclination_deg(sat)

        for (t_start, t_end) in monday_windows_utc:

            tiempos_eventos, tipos_eventos = sat.find_events(
                observer_location, t_start, t_end, altitude_degrees=0.0
            )

            pase_actual = {}
            max_elev_deg = 0.0
            aos_local_dt = None

            for (t_evento, tipo_evento) in zip(tiempos_eventos, tipos_eventos):
                dt_utc = t_evento.utc_datetime().replace(tzinfo=datetime.timezone.utc)
                dt_local = dt_utc.astimezone(LOCAL_TZ)

                if tipo_evento == 0:
                    pase_actual = {}
                    pase_actual['objeto'] = sat_name
                    aos_local_dt = dt_local
                    pase_actual['aos_local_dt'] = aos_local_dt
                    pase_actual['aos_local_str'] = dt_local.strftime('%Y-%m-%d %H:%M:%S')
                    max_elev_deg = 0.0

                elif tipo_evento == 1:
                    difference = sat - observer_location
                    topocentrico = difference.at(t_evento)
                    alt, az, distance = topocentrico.altaz()
                    max_elev_deg = alt.degrees


                    geocentrico = sat.at(t_evento)
                    dist_centro_tierra = geocentrico.distance().km
                    altitud_km = dist_centro_tierra - 6378.0
                    pase_actual['altitud_km'] = f"{altitud_km:.1f}"

                elif tipo_evento == 2:
                    if aos_local_dt is not None:
                        pase_actual['los_local_str'] = dt_local.strftime('%Y-%m-%d %H:%M:%S')
                        pase_actual['inclinacion_deg'] = f"{incl_deg:.2f}"


                        if MIN_ELEV_DEG <= max_elev_deg <= MAX_ELEV_DEG:

                            info_satelite = {
                                "descripcion": "No disponible",
                                "frecuencia": "Desconocida",
                                "modo": "Desconocido",
                            }
                            for known_name, satdata in SAT_INFO.items():

                                if known_name.upper() in sat_name.upper():
                                    info_satelite = satdata
                                    break

                            pase_actual['descripcion'] = info_satelite.get("descripcion", "No disponible")
                            pase_actual['frecuencia'] = info_satelite.get("frecuencia", "Desconocida")
                            pase_actual['modo'] = info_satelite.get("modo", "Desconocido")
                            pase_actual['elev_max_deg'] = f"{max_elev_deg:.1f}"


                            if (aos_local_dt.weekday() == 0) and (18 <= aos_local_dt.hour < 21):
                                resultados.append(pase_actual)

                    pase_actual = {}
                    aos_local_dt = None


    resultados.sort(key=lambda x: x['aos_local_dt'])

    if not resultados:
        print("No se encontraron pases los próximos 3 lunes entre 18:00 y 21:00 (hora local)"
              f" con elevación {MIN_ELEV_DEG}°–{MAX_ELEV_DEG}°.")
        return

    print("Pases encontrados en los próximos 3 lunes (18:00–21:00 local), elev. máx. "
          f"{MIN_ELEV_DEG}°–{MAX_ELEV_DEG}°:\n")

    encabezado = (
        "{:<35} | {:<30} | {:<8} | {:<8} | {:<8} | {:<15} | {:<10} | {}"
    )
    print(encabezado.format(
        "Objeto",
        "Rango Horario Local [AOS→LOS]",
        "Alt(km)",
        "Inc.(°)",
        "MaxEl",
        "Frecuencia",
        "Modo",
        "Descripción"
    ))
    print("-" * 150)

    for p in resultados:
        aos_str = p.get('aos_local_str', 'N/A')
        los_str = p.get('los_local_str', 'N/A')

        try:
            date_aos = aos_str.split(' ')[0]
            time_aos = aos_str.split(' ')[1] if ' ' in aos_str else ''
            date_los = los_str.split(' ')[0]
            time_los = los_str.split(' ')[1] if ' ' in los_str else ''

            if date_aos == date_los:
                rango_horario_local = f"{date_aos} {time_aos} → {time_los}"
            else:
                rango_horario_local = f"{aos_str} → {los_str}"
        except:
            rango_horario_local = f"{aos_str} → {los_str}"

        print(encabezado.format(
            p.get('objeto', 'N/A'),
            rango_horario_local,
            p.get('altitud_km', 'N/A'),
            p.get('inclinacion_deg', 'N/A'),
            p.get('elev_max_deg', 'N/A'),
            p.get('frecuencia', 'N/A'),
            p.get('modo', 'N/A'),
            p.get('descripcion', 'N/A'),
        ))

if __name__ == "__main__":
    main()


Descargando TLE desde: https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle
TLE descargado y guardado en 'satellites.tle'.

Cargando TLE desde fichero local...

Se han cargado 11020 satélites.

Pases encontrados en los próximos 3 lunes (18:00–21:00 local), elev. máx. 70.0°–90.0°:

Objeto                              | Rango Horario Local [AOS→LOS]  | Alt(km)  | Inc.(°)  | MaxEl    | Frecuencia      | Modo       | Descripción
------------------------------------------------------------------------------------------------------------------------------------------------------
STARLINK-2751                       | 2025-02-17 18:00:01 → 18:12:44 | 541.3    | 53.05    | 80.8     | Ku/Ka-band      | Datos      | Constelación de internet banda ancha (SpaceX).
KINEIS-5E                           | 2025-02-17 18:00:05 → 18:13:25 | 644.9    | 97.95    | 74.0     | Desconocida     | Desconocido | No disponible
STARLINK-2299                       | 2025-02-17 18:00:19 → 18:13:01 | 54